In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py, time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mat
import seaborn as sb
import tensorflow as tf
import scipy

import sys
sys.path.append('../../../..')
import mutagenesisfunctions as mf
import bpdev as bd
import helper
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency

import contacts

from Bio import AlignIO
import time as time
import pandas as pd
np.random.seed(42)

In [28]:
fam = 'RF00059'
routfile = '%s_red.sorted.out'%(fam)
ritefile = '%s_rscores.txt'%(fam)

#extract the lines we want from the outfile
ritelines = []
with open(os.path.join('rscape_scores', routfile), 'r') as fd:
    start = False
    done = False
    for line in fd:
        if not start:
            if 'left_pos' in line:
                start = True
                ritelines.append(line)
            else:
                continue
        elif start:
            ritelines.append(line)
            if 'cyk-cov' in line:
                done = True
        if done:
            break


In [29]:
#write them to a temp file
with open(os.path.join('rscape_scores', ritefile), 'w') as fd:
    for line in ritelines:
        fd.write(line)

In [30]:
#read it in as a csv
rscores = pd.read_csv(os.path.join('rscape_scores', ritefile), delim_whitespace=True).iloc[1:-1, 1:]
rscores

,left_pos,right_pos,score,E-value,substitutions,power
1,67,78,19294.43807,1.64326e-52,14357.0,1.0
2,9,89,17270.33499,1.53613e-46,16678.0,1.0
3,8,90,15419.46048,4.39565e-41,16700.0,1.0
4,68,77,14731.14512,4.60687e-39,9906.0,1.0
5,10,88,14122.49680,2.93747e-37,14944.0,1.0
6,11,54,12340.52889,5.06618e-32,15185.0,1.0
7,7,91,12131.55590,2.10723e-31,17146.0,1.0
8,38,47,12087.29000,2.87262e-31,17171.0,1.0
9,40,45,11223.53414,1.00246e-28,17287.0,1.0
10,39,46,11062.28800,2.96376e-28,16171.0,1.0
